In [1]:
import roslib
import rosbag
import numpy as np
import time, sys, os
from ros import rosbag
import roslib, rospy
roslib.load_manifest('sensor_msgs')
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import subprocess
import re

In [2]:
#  For correct sorting.
def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [atoi(c) for c in re.split('(\d+)', text)]


In [3]:
def CreateRosbagFromVideo(videopath, bag,topic,encod,bag_frame_id):  
    '''Add video file topic to rosbag'''
    frame_id = 0
    cap = cv2.VideoCapture(videopath)
    cb = CvBridge()
    ret = True
    
    while(ret):
        ret, frame = cap.read()        
        if not ret:
            break

        if encod=='mono8':
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        stamp = rospy.rostime.Time.from_sec(0)  
        stamp.secs = stamp.secs + 1341836530 + (frame_id)                                  

        image = cb.cv2_to_imgmsg(frame, encoding=encod)
        image.header.stamp = stamp
        image.header.frame_id = bag_frame_id

        bag.write(topic, image, stamp)       
        frame_id += 1
    cap.release()

In [4]:
def createRosbagFromImages(folderPath,bag,topic,encod,bag_frame_id):    
    frames = os.listdir(folderPath+'/')    
    frames.sort(key=natural_keys)
    cb = CvBridge()
    
    height, width, layers = cv2.imread(folderPath+'/'+frames[0]).shape
#   I realized that the pictures should be about size 640x480, I used in 640X360. 
    isResize = height > 640 or width > 360
    
    for frame_id,frame_path in enumerate(frames):  
        frame = cv2.imread(folderPath+'/'+frame_path)

        if isResize:
            frame = cv2.resize(frame, (360, 640))

        if encod=='mono8':
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if isResize:
#               If you change the size of the image,
#               you need to pass a filter to arrange a binary image for segmentation images.
                ret,frame = cv2.threshold(frame,200,255,0)

        stamp = rospy.rostime.Time.from_sec(0)  
        stamp.secs = stamp.secs + 1341836530 + (frame_id)                                  

        image = cb.cv2_to_imgmsg(frame, encoding=encod)
        image.header.stamp = stamp
        image.header.frame_id = bag_frame_id

        bag.write(topic, image, stamp)   
    print frame_id

In [16]:
# create rosbag from videos
name = "../../src/rosbag2video/galaxy360.bag"
with rosbag.Bag(name, 'w') as outbag:
    CreateRosbagFromVideo('../../src/data/Video/galaxy360_rotate.avi',outbag,'/rect_color_raw','bgr8','camera_rgb_optical_frame')
    CreateRosbagFromVideo('../../src/data/Video/galaxy360_segmentation.avi',outbag,'/cnn_label','mono8','openni_rgb_optical_frame')    

(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 360, 3)
(640, 

In [7]:
# create rosbag from videos
newRosBagName = "../../src/rosbag2video/almostrectangle.bag"
with rosbag.Bag(newRosBagName, 'w') as outbag:
    createRosbagFromImages('../../src/data/Images/almostrectangle/almostrectangle_orginal/',outbag,'/rect_color_raw','bgr8','camera_rgb_optical_frame')
    createRosbagFromImages('../../src/data/Images/almostrectangle/almostrectangle_segmentation/',outbag,'/cnn_label','mono8','openni_rgb_optical_frame')    

200
200
mv ../../src/rosbag2video/almostrectangle.bag ../../../Documents/popup_ws/src/pop_up_slam/pop_planar_slam/dataset/
